In [2]:
pip install selenium

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.0 MB 1.7 MB/s eta 0:00:06
    --------------------------------------- 0.2/10.0 MB 2.0 MB/s eta 0:00:06
    --------------------------------------- 0.2/10.0 MB 1.7 MB/s eta 0:00:06
    --------------------------------------- 0.2/10.0 MB 1.7 MB/s eta 0:00:06
   - -------------------------------------- 0.4/10.0 MB 1.5 MB/s eta 0:00:07
   -- ------------------------------------- 0.5/10.0 MB 1.8 MB/s eta 0:00:06
   -- ------------------------------------- 0.5/10.0 MB 1.8 MB/s eta 0:00:06
   -- ------------------------------------- 0.6/10.0 MB 1.6 MB/s eta 0:00:07
   --- ------------------------------------ 0.8/10.0 MB 1.7 MB/s eta 0:00:06
   --- ------------------------------------ 0.8/10.0 MB 1.7 MB/s eta 0:00:06
   --- ------------------------------------ 0.9/10.0 MB 1.7 MB/s eta 0:00:06
   ---- ----------------------------------- 1.1/10.0 MB 1.9 MB/s eta 0:00:05
   ---

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.2 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import libraries yang dibutuhkan
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [6]:
# Inisialisasi WebDriver
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
url = "https://www.carsome.id/en/buy-car" # URL target yang akan discraping
driver = webdriver.Chrome(options=options)
driver.get(url)

In [7]:
# Menentukan CSS selector untuk pop-up dan tombol close
popup_selector = "div.location-model-box"
close_button_selector = "img.location-model-box__head-close"

# Tunggu hingga pop-up muncul dan temukan tombol close
try:
    popup = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, popup_selector)))
    close_button = popup.find_element(By.CSS_SELECTOR, close_button_selector)
    close_button.click()  # Klik tombol close
except (NoSuchElementException, TimeoutException):
    pass  # Jika pop-up tidak ditemukan, lanjutkan scraping

In [8]:
# Fungsi untuk mengekstrak informasi mobil dari setiap item
def extract_car_info(item):
    car_info = {}

    title_elem = item.select_one('.mod-b-card__title')
    if title_elem:
        # Mendapatkan teks dari elemen '.mod-b-card__title'
        title_text = title_elem.text.strip()
        
        # Memecah teks judul berdasarkan karakter newline
        title_parts = title_text.split('\n')
        
        # Mendapatkan merek (brand) dari bagian judul
        brand = title_parts[1].strip()  # Mendapatkan merek dari bagian kedua
        car_info['brand'] = brand
        
        # Mendapatkan model dari bagian judul
        model = ' '.join(title_parts[2:])  # Menggabungkan semua elemen kecuali yang pertama (tahun)
        car_info['model'] = model.strip()   # Menghapus spasi ekstra
        
        # Mendapatkan tahun dari bagian judul
        year = title_parts[0].strip()  # Mengambil tahun dari bagian pertama
        car_info['year'] = year
    else:
        car_info['year'] = 'N/A'
        car_info['model'] = 'N/A'

    # Mendapatkan mileage dari elemen '.mod-b-card__car-other'
    mileage_elem = item.select_one('.mod-b-card__car-other span:nth-of-type(1)')
    if mileage_elem:
        car_info['mileage'] = mileage_elem.text.strip()
    else:
        car_info['mileage'] = 'N/A'

    # Mendapatkan jenis transmisi dari elemen '.mod-b-card__car-other'
    transmission_elem = item.select_one('.mod-b-card__car-other span:nth-of-type(2)')
    if transmission_elem:
        car_info['transmission'] = transmission_elem.text.strip()
    else:
        car_info['transmission'] = 'N/A'

    # Mendapatkan lokasi seller dari elemen '.mod-b-card__car-other'
    location_elem = item.select_one('.mod-b-card__car-other span:nth-of-type(3)')
    if location_elem:
        car_info['seller_location'] = location_elem.text.strip()
    else:
        car_info['seller_location'] = 'N/A'

    # Mendapatkan harga dari elemen '.mod-card__price-cash'
    price_elem = item.select_one('.mod-card__price-cash')
    if price_elem:
        car_info['price'] = price_elem.text.strip()
    else:
        car_info['price'] = 'N/A'

    # Mendapatkan cicilan dari elemen '.mod-tooltipMonthPay'
    installment = item.select_one('.mod-tooltipMonthPay')
    if installment:
        car_info['installment'] = installment.text.strip()
    else: 
        car_info['installment'] = 'N/A'
        
    return car_info # Mengembalikan hasil ekstrasi sebagai sebuah dictionary

In [9]:
# Looping untuk melalukan scraping di beberapa halaman
products = []
while True:
    time.sleep(10)  # Tunggu 10 detik sebelum melanjutkan ke halaman berikutnya (bisa disesuaikan sesuai kecepatan internet)
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "button.v-pagination__navigation[aria-label='Next page']")
        if next_button.is_enabled():
            next_button.click()  # Klik tombol halaman berikutnya
            time.sleep(10)  # Tunggu halaman dimuat sepenuhnya
        else:
            break  # Keluar dari loop jika tidak ada lagi halaman berikutnya
    except NoSuchElementException:
        break  # Keluar dari loop jika tidak ada tombol navigasi
    except TimeoutException:
        break  # Keluar dari loop jika waktu tunggu sudah habis
    
    # Parsing HTML dari halaman yang baru dimuat
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # Ekstrak informasi mobil dari setiap item
    for item in soup.select('div.list-card__item'):
        car_info = extract_car_info(item)
        products.append(car_info)
        
# Tutup WebDriver
driver.quit()

In [19]:
# Mengubah hasil scrapping menjadi dataframe
carsome_scrapping_df = pd.DataFrame(products)

# Menghapus baris berisi nilai null
carsome_scrapping_df.dropna(inplace= True)

# Eksport dataframe sebagai file csv
carsome_scrapping_df.to_csv('carsome_scrapping.csv', index= False)

# Menampilkan dataframe
print(carsome_scrapping_df)

          brand                                              model  year  \
0      Daihatsu                         AYLA         R         1.2  2020   
1        Toyota                        YARIS         G         1.5  2019   
2      Daihatsu                       TERIOS         X         1.5  2015   
3      Daihatsu                         AYLA         R         1.2  2019   
4         Honda                     CR-V         TURBO         1.5  2017   
..          ...                                                ...   ...   
131  Mitsubishi                  XPANDER         SPORT         1.5  2018   
132       Honda                   BRIO         SATYA E         1.2  2019   
134       Honda                        BRIO         RS         1.2  2018   
135      Nissan  GRAND LIVINA         HIGHWAY STAR AUTECH      ...  2017   
136      Toyota                       SIENTA         V         1.5  2016   

        mileage transmission    seller_location                  price  \
0     54.801 